In [1]:
import numpy as np
import pickle
import galsim
# import afw psf options

In [2]:
with open('psf_test.pkl', 'rb') as file:
    psf = pickle.load(file)

print(psf)
print(psf.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(35, 35)


In [ ]:
ipsf = galsim.Image(psf)
gpsf = galsim.InterpolatedImage(ipsf, scale=0.2, normalization='flux')
print(gpsf)
print(gpsf.calculateMomentRadius()/0.2) #why is this slightly smaller than measued value from DP1?
print(gpsf.calculateFWHM()/0.2)
pixel = galsim.Pixel(0.2)
epsf = galsim.Convolve(pixel, gpsf)
print(epsf.calculateFWHM()/0.2)

galsim.InterpolatedImage(image=galsim.Image(bounds=galsim.BoundsI(-17,17,-17,17), scale=0.2, dtype=numpy.float64), flux=0.99999999611243)
2.7716448140692163
4.156009037928731
4.225404329113469


In [ ]:
dpsf = galsim.InterpolatedImage(ipsf, scale=0.2, depixelize=True, normalization='flux') #this can account for pixel response, does it affect ellipticity?
print(dpsf.calculateFWHM()/0.2)
pixel = galsim.Pixel(0.2)
epsf = galsim.Convolve(pixel, dpsf)
print(epsf.calculateFWHM()/0.2)

4.160901013601039


In [15]:
psf_optical = galsim.Convolve(galsim.Deconvolve(pixel),gpsf) # what does it mean to deconvolve pixel here? is this safe with noise
print(psf_optical.calculateFWHM()/0.2)
final_obj = galsim.Convolve([pixel, psf_optical])
print(final_obj.calculateFWHM()/0.2)

4.098703851172976
4.163275803956206
